In [1]:
# Import the necessary modules
import numpy as np
import pandas as pd
import datetime as dt
from dateutil import parser
#Plotting 
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set(style='ticks', palette='RdBu')
%matplotlib inline


In [2]:
# Import the data files and assign them to DataFrames
broker_df=pd.read_csv("Data/broker.csv")
lob_df=pd.read_csv("Data/line_of_business.csv")
policy_item_df=pd.read_csv("Data/policy_item.csv")
policy_df=pd.read_csv("Data/policy.csv")


In [3]:
# Have a look:
# policy_item_df.isnull().sum()
# print(policy_item_df.dtypes)
policy_df.isnull().sum() #check how many missing value there are

policy_id                0
policy_holder_name       0
policy_holder_surname    0
policy_effective_date    3
policy_broker_id         0
dtype: int64

In [4]:
# Clean the data up
# Answer 1
policy_item_df.fillna(0, inplace=True)
policy_item_df.isnull().sum() # confirm it worked

policy_df['policy_effective_date'].fillna('1900-01-31', inplace=True)
policy_df.isnull().sum() #check how many missing value there are


policy_id                0
policy_holder_name       0
policy_holder_surname    0
policy_effective_date    0
policy_broker_id         0
dtype: int64

In [5]:
# Total policy insured including VAT
#Answer 2
policy_item_df['total_policy_insured_incl_vat']=policy_item_df['policy_insured_value_excl_vat']+policy_item_df['policy_insured_value_vat']
policy_item_df.head(3)


,policy_item_id,policy_id,policy_insured_value_excl_vat,policy_insured_value_vat,broker_fee_excl_vat,broker_fee_vat,line_of_business_id,total_policy_insured_incl_vat
0,c5e4987c-6c6b-4a73-aebc-be91515fb2b4,8045a7ac-8532-4d3b-9a5f-44508050ba1f,115087.0,17263.05,23017.4,3452.61,0,132350.05
1,fc8b51ea-cd7f-4f85-ac05-99b2364868ba,8045a7ac-8532-4d3b-9a5f-44508050ba1f,24874.0,3731.10,4974.8,746.22,1,28605.10
2,c81b40ab-4ee9-4408-bade-c5a3e0ca5a1c,8045a7ac-8532-4d3b-9a5f-44508050ba1f,133518.0,20027.70,26703.6,4005.54,2,153545.70


In [6]:
# Define a function to parse the various date formats
def clean_date(date):
  """Function to clean and standardize the input date. Output is in the format YYYY-MM-DD."""
  try:
    d=parser.parse(date, fuzzy=True, yearfirst=True)
    return d.date()

  except (OverflowError, ParserError ) as e:
    print("Error: {} dealing with date {}".format(e, date))

In [7]:
# Answer 4
# Apply date cleaning function. parser.parse needs to recieve string so cast the column as such. 
policy_df['policy_effective_date']=policy_df['policy_effective_date'].astype(str).apply(clean_date)
policy_df.head()

,policy_id,policy_holder_name,policy_holder_surname,policy_effective_date,policy_broker_id
0,fe978e90-5c0e-4375-9b37-0f3b55bfddd3,Ryland,Sanchez,2021-05-24,2
1,3954e8ef-0d81-4951-ad15-969fba099da5,Kela,Carter,2020-06-02,3
2,b72a1596-8258-4df8-839a-434e0fc39b5a,Cherish,King,2020-08-31,1
3,20354db3-30db-4cb3-b6b4-b301128b06c8,Basim,Taylor,2021-06-12,5
4,f5d7bcf5-addc-4955-afec-ef2d6c38f986,Kylah,Hall,2021-01-08,9


In [8]:
# Join policy with policy item:
pol_comb_df=policy_item_df.set_index('policy_id').join(policy_df.set_index('policy_id'), on='policy_id')
# Answer 3-What is the total policy insured including vat per broker?
total_by_broker_df=pol_comb_df[['total_policy_insured_incl_vat','policy_broker_id']].groupby(['policy_broker_id']).sum()
highest_broker=total_by_broker_df[total_by_broker_df['total_policy_insured_incl_vat']==total_by_broker_df['total_policy_insured_incl_vat'].max()]

total_by_broker_df

,total_policy_insured_incl_vat
policy_broker_id,
1,2.219249e+08
2,2.148883e+08
3,1.906467e+08
4,1.827889e+08
5,1.993789e+08
6,2.206877e+08
7,2.335157e+08
8,2.193348e+08
9,2.313410e+08


In [9]:
# 5: Which policies have the most policy items (Join & Filter)
#Create new Dataframe, grouping by policy ID and aggregating with count
pol_count_df=pol_comb_df.groupby(['policy_id']).agg(num_policies=('policy_item_id', 'count')).sort_values(by='num_policies', ascending=False)
#Select highest policy counts
hi_pol_count_df=pol_count_df[pol_count_df['num_policies']==pol_count_df['num_policies'].max()]
hi_pol_count_df


,num_policies
policy_id,
3a46b5a8-6dc5-4ea2-827c-7ebeee3ac356,20
4aa84e76-d0a3-4696-894b-9a25cd5df126,20
5574d058-0cc0-4a2b-8b92-60ac80f79b1c,20
9ac5fb4c-b0a6-4b03-b968-86ff62ff3087,20
bff67223-2578-45eb-acef-b03336dc71ac,20
...,...
a476d002-088e-44b6-b564-ef18bd471bd3,20
42c02a52-468f-4293-b265-978704cc36f4,20
95502e3d-133b-4b76-adb4-9b199095b6fa,20


In [15]:
# 6. Which broker made the most profit? (Join & GroupBy)
temp_df=pol_comb_df.merge(broker_df, left_on='policy_broker_id', right_on='broker_id')
broker_profits_df=temp_df[['broker_name','broker_fee_excl_vat']].groupby('broker_name').sum()
#Find Broker with highest fee (Answer 6)
broker_prof_max_df=broker_profits_df[broker_profits_df['broker_fee_excl_vat']==broker_profits_df['broker_fee_excl_vat'].max()]
broker_prof_max_df

,broker_fee_excl_vat
broker_name,
BrokerG,3.676445e+07


In [11]:
# 7. Which Line of Business is the most profitable? (Join & GroupBy)
profit_lob_df=pol_comb_df[['broker_fee_excl_vat', 'line_of_business_id']].groupby('line_of_business_id').sum()
temp2=profit_lob_df.merge(lob_df, on='line_of_business_id')

most_prof_lob_df=temp2[temp2['broker_fee_excl_vat']==temp2['broker_fee_excl_vat'].max()]
most_prof_lob_df


,line_of_business_id,broker_fee_excl_vat,line_of_business_description
0,1,3.075060e+07,Appliances


In [12]:
# 8. Which broker made the most money per line of business? (Join & Pivot)
t=pol_comb_df[['policy_broker_id', 'line_of_business_id', 'broker_fee_excl_vat']].groupby(by=['policy_broker_id', 'line_of_business_id']).sum()

t.reset_index(inplace=True)
tp=t.pivot(index='policy_broker_id', columns='line_of_business_id', values='broker_fee_excl_vat')


pd.options.display.max_colwidth = 700
cm = sns.color_palette("flare", as_cmap=True)
# tp.style.format("{:.2f}").background_gradient(cmap=cm)
tp.style.highlight_max(color='blue').format("{:.2f}")
# tp.max()

line_of_business_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
policy_broker_id,,,,,,,,,,,,,,,,,,,,
1,3412132.82,3069776.49,2723956.29,2919711.54,2608665.44,2360712.95,2229887.39,2183597.71,1988618.42,1877439.93,1595605.20,1326487.96,1439008.70,1218377.50,1048721.59,892831.85,694647.85,516830.04,408293.61,104315.52
2,3290186.88,3072232.57,2862272.46,2855265.82,2470388.83,2578790.79,2374923.05,2241842.10,1679274.70,1728241.02,1512375.25,1373620.95,1343514.46,1147569.40,977969.12,798034.84,638295.51,397001.20,270266.61,152879.58
3,3023967.93,2760618.62,2804190.57,2415466.07,2224140.84,2074433.75,1875334.51,1789600.38,1644853.64,1600826.96,1206773.66,1394992.30,1273660.36,876171.58,794019.94,674658.18,624746.85,326844.77,303628.74,215422.77
4,2860279.19,2471518.80,2565640.66,2287816.54,2230991.62,1971803.51,2048643.30,1679407.88,1731605.51,1641401.28,1328614.72,1256315.91,1074211.49,855130.41,800386.17,703655.69,498268.62,340728.34,312518.01,93781.60
5,2984824.63,2899562.00,2648790.33,2585692.76,2298697.54,2329417.67,2306093.93,1948678.02,1770784.94,1665041.59,1504293.79,1308151.65,1021450.79,917266.88,836540.02,692405.61,584532.42,408047.66,270589.76,160843.30
6,3266212.97,3286049.87,2909087.24,3034548.51,2562168.59,2604583.20,2490803.48,2324839.70,1919506.66,1595027.18,1414574.07,1450969.78,1250499.87,1123342.63,941954.29,834367.79,481203.54,399554.02,240563.83,137785.60
7,3662806.02,3419750.76,3243688.90,3000348.74,2730077.48,2291145.30,2641723.27,2355310.75,2059761.97,2058573.71,1763578.49,1454062.71,1426580.86,1133728.56,1113231.24,699581.94,597403.95,565899.05,355706.32,191488.35
8,3522655.37,3147734.76,2779266.09,2791699.13,2883033.90,2416177.21,2511119.98,2269906.62,2005238.50,1962371.28,1611026.60,1345294.17,1150913.61,967638.88,934091.35,713226.72,562285.72,470050.33,321373.60,97450.21
9,3635305.94,3405911.74,3187069.55,3015580.63,2722579.82,2421373.73,2321570.70,2226646.68,1936025.01,1732495.29,1800225.46,1596238.35,1437965.62,1156087.81,1192452.97,827527.60,679909.51,551336.50,395518.25,243475.06


In [13]:
# Effective date with the largest policy insured including VAT (Answer 9)
# By Broker and Date
broker_by_date_df=pol_comb_df.set_index(['policy_effective_date', 'policy_broker_id'])[['total_policy_insured_incl_vat']].groupby(['policy_effective_date','policy_broker_id']).sum()
broker_by_date_df.loc[broker_by_date_df.idxmax()]

,,total_policy_insured_incl_vat
policy_effective_date,policy_broker_id,
2020-04-08,9,5260178.2


In [14]:
# By Date only
date_total_df=pol_comb_df.reset_index()[['policy_effective_date', 'total_policy_insured_incl_vat']].groupby(['policy_effective_date']).sum()
date_total_df.loc[date_total_df.idxmax()]

,total_policy_insured_incl_vat
policy_effective_date,
2020-09-28,9578869.8
